# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** Jakub Bednarek<br>
**Kontakt:** jakub.bednarek@put.poznan.pl<br>
**Materiały:** [Strona WWW](http://jakub.bednarek.pracownik.put.poznan.pl)

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Cel ćwiczeń:
- zapoznanie się z rekurencyjnymi sieciami neuronowymi,
- stworzenie modelu sieci z warstwami rekurencyjnymi dla zbioru danych MNIST,
- stworzenie własnych implementacji warstwami neuronowych

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np

TensorFlow 2.x selected.


In [0]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, LSTM, LSTMCell, SimpleRNNCell
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, RMSprop
from tensorflow.python.keras import backend as K


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.astype('float32')  # shape: 60000, 28, 28
x_test = x_test.astype('float32')    # shape: 10000, 28, 28
x_train /= 255  # normalizacja wartości do przedziału [0, 1]
x_test /= 255

y_train = to_categorical(y_train, 10)  # zamiana etykiety na one-hot encoding; np. 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_test = to_categorical(y_test, 10)

## Sieci rekurencyjne
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://www.tensorflow.org/guide/keras/rnn

https://www.tensorflow.org/guide/function

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Przykładowy model z warstwą rekurencyjną dla danych MNIST:

In [0]:
class RecurrentModel(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm_1 = LSTM(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.lstm_1(inputs)
        return self.dense_1(x)

model = RecurrentModel(num_classes=10)

In [5]:
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 55s 920us/sample - loss: 0.5992 - accuracy: 0.8227
Epoch 2/2
60000/60000 [==============================] - 55s 910us/sample - loss: 0.1541 - accuracy: 0.9587


### Zadanie 1
Rozszerz model z powyższego przykładu o kolejną warstwę rekurencyjną przed gęstą warstwą wyjściową.

Standardowe sieci neuronowe generują jeden wynik na podstawie jednego inputu.
Natomiast sieci rekurencyjne przetwarzają dane sekwencyjnie, w każdym kroku łącząc wynik poprzedniego przetwarzania i aktualnego wejścia. Dlatego domyślnym wejściem sieci neuronowej jest tensor 3-wymiarowy ([batch_size,sequence_size,sample_size]).
Domyślnie warstwy rekurencyjne w Kerasie zwracają tylko wyniki przetwarzania ostatniego
kroku (otrzymują tensor 3-wymiarowy, zwracają tensor 2-wymiarowy). Jeśli chcesz zwrócić sekwencje wyników wszystkich kroków przetwarzania dla warstwy rekurencyjnej, musisz ustawić parametr return_sequences=True.


In [6]:
class DoubleRecurrentModel(Model):

    def __init__(self, num_classes=10):
        super(DoubleRecurrentModel, self).__init__(name='double_recurrent_model')
        self.num_classes = num_classes
        self.lstm_1 = LSTM(128, activation='relu', return_sequences=True)
        self.lstm_2 = LSTM(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')
        

    def call(self, inputs):
        x = self.lstm_1(inputs)
        x = self.lstm_2(x)
        return self.dense_1(x)

recurrentModel = DoubleRecurrentModel()

recurrentModel.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

recurrentModel.fit(x_train, y_train, batch_size=32, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 119s 2ms/sample - loss: 0.5585 - accuracy: 0.8294
Epoch 2/2
60000/60000 [==============================] - 118s 2ms/sample - loss: 0.1322 - accuracy: 0.9643


### Zadanie 2 
Wykorzystując model z przykładu, napisz sieć rekurencyjną przy użyciu SimpleRNNCell.

Cell implementuje tylko operacje wykonywane przez warstwę
rekurencyjną dla jednego kroku. Warstwy rekurencyjne w każdym kroku
łączą wynik operacji poprzedniego kroku i aktualny input.
Wykorzystaj pętle for do wielokrotnego wywołania komórki SimpleRNNCell (liczba kroków to liczba elementów w sekwencji). Aby wywołać SimpleRNNCell dla pojedynczego wejścia i stanu należy użyć jej metody ```call``` analogicznie jak w przypadku własnych modeli (tzn. ```my_model(input)```). 



Wywołanie zainicjalizowanej komórki rekurencyjnej wymaga podania aktualnego inputu i listy stanów ukrytych poprzedniego kroku (SimpleRNNCell ma jeden stan).

Trzeba zainicjalizować ukryty stan warstwy z wartościami początkowymi (można wykorzystać rozkład normalny - tf.random.normal).

In [7]:
class SimpleRNNCellModel(Model):

    def __init__(self, num_classes=10):
        super(SimpleRNNCellModel, self).__init__(name='simple_rnn_cell_model')
        self.num_classes = num_classes

        self.simple_rnn_cell = SimpleRNNCell(128, activation='relu')
        self.dense = Dense(self.num_classes, activation='softmax') 

    def call(self, inputs):
        sequence = [tf.random.normal([32,128])]
        for i in range(28):
            out, sequence = self.simple_rnn_cell(inputs[:,i,:], sequence)
        return self.dense(out)

simpleRNNCellModel = SimpleRNNCellModel()

simpleRNNCellModel.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

simpleRNNCellModel.fit(x_train, y_train, batch_size=32, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 14s 231us/sample - loss: 0.4967 - accuracy: 0.8415
Epoch 2/2
60000/60000 [==============================] - 13s 218us/sample - loss: 0.2231 - accuracy: 0.9362


### Zadanie 3
Zamień komórkę rekurencyjną z poprzedniego zadania na LSTMCell (LSTMCell ma dwa stany ukryte).

In [8]:
class LSTMCellModel(Model):

    def __init__(self, num_classes=10):
        super(LSTMCellModel, self).__init__(name='lstm_cell_model')
        self.num_classes = num_classes

        self.lstm_cell = LSTMCell(128, activation='relu')
        self.dense = Dense(self.num_classes, activation='softmax') 

    def call(self, inputs):
        sequence = [tf.random.normal([32,128]), tf.random.normal([32, 128])]
        for i in range(28):
            out, sequence = self.lstm_cell(inputs[:,i,:], sequence)
        return self.dense(out)

lstmCellModel = LSTMCellModel()

lstmCellModel.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

lstmCellModel.fit(x_train, y_train, batch_size=32, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 48s 799us/sample - loss: 0.6139 - accuracy: 0.8148
Epoch 2/2
60000/60000 [==============================] - 46s 764us/sample - loss: 0.1371 - accuracy: 0.9602


### Zadanie 4
Wykorzystując model z poprzedniego zadania, stwórz model sieci
neuronowej z własną implementacją prostej warstwy rekurencyjnej.
- w call zamień self.lstm_cell_layer(x) na wywołanie własnej metody np. self.cell(x)
- w konstruktorze modelu usuń inicjalizację komórki LSTM i zastąp ją inicjalizacją warstw potrzebnych do stworzenia własnej komórki rekurencyjnej,
- stwórz metodę cell() wykonującą operacje warstwy rekurencyjnej,
- prosta warstwa rekurencyjna konkatenuje poprzedni wyniki i aktualny input, a następnie przepuszcza ten połączony tensor przez warstwę gęstą (Dense).

In [9]:
class CustomCellModel(Model):

    def __init__(self, num_classes=10):
        super(CustomCellModel, self).__init__(name='custom_cell_model')
        self.num_classes = num_classes

        self.dense_1 = Dense(128, activation='relu')
        self.dense_2 = Dense(self.num_classes, activation='softmax') 

    def call(self, inputs):
        sequence = tf.random.normal([32,128])
        for i in range(28):
            sequence = self.cell(inputs[:,i,:], sequence)
        return self.dense_2(sequence)
    
    def cell(self, inputs, sequence):
        out = K.concatenate([sequence, inputs], 1)
        return self.dense_1(out)

customCellModel = CustomCellModel()

customCellModel.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

customCellModel.fit(x_train, y_train, batch_size=32, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 15s 252us/sample - loss: 0.5512 - accuracy: 0.8207
Epoch 2/2
60000/60000 [==============================] - 14s 230us/sample - loss: 0.2203 - accuracy: 0.9376


### Zadanie 5

Na podstawie modelu z poprzedniego zadania stwórz model z własną implementacją warstwy LSTM. Dokładny i zrozumiały opis działania warstwy LSTM znajduje się na [stronie](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [10]:
class CustomLSTMModel(Model):

    def __init__(self, num_classes=10):
        super(CustomLSTMModel, self).__init__(name='custom_lstm_model')
        self.num_classes = num_classes

        self.dense_1 = Dense(128, activation='sigmoid')
        self.dense_2 = Dense(128, activation='sigmoid')
        self.dense_3 = Dense(128, activation='tanh')
        self.dense_4 = Dense(self.num_classes, activation='softmax') 

    def call(self, inputs):
        sequence = tf.random.normal([32,128])
        for i in range(28):
            sequence = self.cell(inputs[:,i,:], sequence)
        return self.dense_4(sequence)
    
    def cell(self, inputs, h_prev):
        x_1 = K.concatenate([h_prev, inputs], 1)
        z = self.dense_1(x_1)
        r = self.dense_2(x_1)
        x_2 = K.concatenate([r * h_prev, inputs], 1)
        h = self.dense_3(x_2)
        return (1 - z) * h_prev + z * h

customLSTMModel = CustomLSTMModel()

customLSTMModel.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

customLSTMModel.fit(x_train, y_train, batch_size=32, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 35s 589us/sample - loss: 0.3183 - accuracy: 0.8973
Epoch 2/2
60000/60000 [==============================] - 32s 541us/sample - loss: 0.0897 - accuracy: 0.9724
